In [ ]:
%load pileup_analysis.py

### Set directory to mcools

In [1]:
file_path = "" # path to mcools
mcool_files = [""] # list of mcool files
resolution = 5000 # resolution for analysis

### Compute expected

In [ ]:
# Get chr arms
# Use bioframe to fetch the genomic features from the UCSC.
mm39_chromsizes = bioframe.fetch_chromsizes('mm39')
mm39_cens = bioframe.fetch_centromeres('mm39')
mm39_arms = bioframe.make_chromarms(mm39_chromsizes, mm39_cens)

# call this to automaticly assign names to chromosomal arms:
mm39_arms = bioframe.make_viewframe(mm39_arms)
# mm39_arms.to_csv('mm39_arms.bed', sep='\t', header=False, index=False) # To use in CLI
mm39_arms

In [ ]:
# Compute expected ... takes some time
expected_df_list = []
output_names = ['expected/' + name.split(".")[0] + '_expected_cis.tsv' for name in mcool_files]
for mcool_file, output_name in zip(mcool_files, output_names):
    curr_cooler = read_cooler(os.path.join(file_path, mcool_file), resolution)
    expected = expected_cis(
        curr_cooler,
        ignore_diags=0,
        view_df=mm39_arms,
        chunksize=1000000)
    expected_df_list.append(expected)
    expected.to_csv(output_name, sep='\t', index=False, header=True) # Let's save to use in CLI

### Load expected

In [ ]:
output_names = ['expected/' + name.split(".")[0] + '_expected_cis.tsv' for name in mcool_files]
expected_df_list = [pd.read_csv(file, sep="\t") for file in output_names]
expected_df_list

### Loop analysis

In [ ]:
loops_path = "" # path to loops
tsv_files = len(mcool_files)*[""] # directory to loop calls
all_pileups = generate_multiple_dot_pileups2(mcool_files, tsv_files, resolution, view_df = mm39_arms, 
                                                  expected=expected_df_list,
                                                 parent_dir=file_path, csv_dir=loops_path)

fg = plotpup.plot(all_pileups, cols='cooler',
                  score=True, cmap='YlOrBr', scale='log', sym=False,
                  height=5, col_order=[file[:-6] for file in mcool_files], despine=False)

plot_dir = "" # output path for plots
name = "" # name for plot output
plt.savefig(os.path.join(plot_dir, name), format='svg')